In [1]:
pip install -r requirements.txt --quiet

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/System/Volumes/Data/home/conda/feedstock_root/build_artifacts/appnope_1707233003401/work'

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import building_ingestion_utils as biu
import modeling_utils as mu
import data_utils as du
import optimization_utils as ou
import weather_ingestion_utils as wiu

directory = 'CU-BEMS_smart_building energy_and_IAQ_data'

dataframes = du.restructure_dataframes(biu.get_dataframes(directory))

DataFrame: 2018Floor1
Total Floor Consumption: Z1_AC1(kW) + Z1_AC2(kW) + Z2_AC1(kW) + Z2_AC2(kW)
Total Zone Consumption: {'Z2': Z2_AC1(kW) + Z2_AC2(kW), 'Z1': Z1_AC1(kW) + Z1_AC2(kW)}
Total AC Consumption by Zone: {'Z2': Z2_AC1(kW) + Z2_AC2(kW), 'Z1': Z1_AC1(kW) + Z1_AC2(kW)}
DataFrame: 2019Floor1
Total Floor Consumption: Z1_AC1(kW) + Z1_AC3(kW) + Z2_AC1(kW)
Total Zone Consumption: {'Z2': Z2_AC1(kW), 'Z1': Z1_AC1(kW) + Z1_AC3(kW)}
Total AC Consumption by Zone: {'Z2': Z2_AC1(kW), 'Z1': Z1_AC1(kW) + Z1_AC3(kW)}


In [3]:
print(f"Total dataframes created: {len(dataframes)}")

for name, df in dataframes.items():
    print(f'DataFrame: {name} loaded with shape: {df.shape}')

Total dataframes created: 14
DataFrame: 2019Floor6 loaded with shape: (424169, 31)
DataFrame: 2019Floor7 loaded with shape: (525600, 31)
DataFrame: 2019Floor5 loaded with shape: (525600, 31)
DataFrame: 2019Floor4 loaded with shape: (525600, 31)
DataFrame: 2019Floor1 loaded with shape: (525600, 13)
DataFrame: 2019Floor3 loaded with shape: (525600, 31)
DataFrame: 2019Floor2 loaded with shape: (525600, 38)
DataFrame: 2018Floor6 loaded with shape: (264960, 31)
DataFrame: 2018Floor7 loaded with shape: (264960, 31)
DataFrame: 2018Floor5 loaded with shape: (264960, 31)
DataFrame: 2018Floor4 loaded with shape: (264960, 31)
DataFrame: 2018Floor1 loaded with shape: (264960, 13)
DataFrame: 2018Floor3 loaded with shape: (264960, 31)
DataFrame: 2018Floor2 loaded with shape: (264960, 38)


In [4]:
du.check_column_names(dataframes)

DataFrame: 2019Floor6
Columns: ['DataFrame Name', 'Date', 'z1_AC1(kW)', 'z1_Light(kW)', 'z1_Plug(kW)', 'z1_S1(degC)', 'z1_S1(RH%)', 'z1_S1(lux)', 'z2_AC1(kW)', 'z2_Light(kW)', 'z2_Plug(kW)', 'z2_S1(degC)', 'z2_S1(RH%)', 'z2_S1(lux)', 'z3_Light(kW)', 'z3_Plug(kW)', 'z4_AC1(kW)', 'z4_AC2(kW)', 'z4_AC3(kW)', 'z4_AC4(kW)', 'z4_Light(kW)', 'z4_Plug(kW)', 'z4_S1(degC)', 'z4_S1(RH%)', 'z4_S1(lux)', 'z5_AC1(kW)', 'z5_Light(kW)', 'z5_Plug(kW)', 'z5_S1(degC)', 'z5_S1(RH%)', 'z5_S1(lux)']

DataFrame: 2019Floor7
Columns: ['DataFrame Name', 'Date', 'z1_AC1(kW)', 'z1_AC2(kW)', 'z1_AC3(kW)', 'z1_AC4(kW)', 'z1_Light(kW)', 'z1_Plug(kW)', 'z1_S1(degC)', 'z1_S1(RH%)', 'z1_S1(lux)', 'z2_AC1(kW)', 'z2_Light(kW)', 'z2_Plug(kW)', 'z2_S1(degC)', 'z2_S1(RH%)', 'z2_S1(lux)', 'z3_Light(kW)', 'z3_Plug(kW)', 'z4_AC1(kW)', 'z4_Light(kW)', 'z4_Plug(kW)', 'z4_S1(degC)', 'z4_S1(RH%)', 'z4_S1(lux)', 'z5_AC1(kW)', 'z5_Light(kW)', 'z5_Plug(kW)', 'z5_S1(degC)', 'z5_S1(RH%)', 'z5_S1(lux)']

DataFrame: 2019Floor5
Columns: [

Data is saved in dictionary of dataframes called `dataframe`

In [5]:
du.column_presence_checker(dataframes)

,DataFrame Name,Date,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(RH%),z1_S1(degC),...,z4_Plug(kW),z4_S1(RH%),z4_S1(degC),z4_S1(lux),z5_AC1(kW),z5_Light(kW),z5_Plug(kW),z5_S1(RH%),z5_S1(degC),z5_S1(lux)
0,2019Floor6,object,float64,Column not present,Column not present,Column not present,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,2019Floor7,object,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2,2019Floor5,object,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
3,2019Floor4,object,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
4,2019Floor1,object,Column not present,Column not present,Column not present,Column not present,float64,float64,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present
5,2019Floor3,object,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
6,2019Floor2,object,float64,Column not present,Column not present,Column not present,float64,float64,float64,float64,...,float64,float64,float64,float64,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present
7,2018Floor6,object,float64,Column not present,Column not present,Column not present,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
8,2018Floor7,object,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
9,2018Floor5,object,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [6]:
dataframes = du.convert_date_columns(dataframes, 'Date')
dataframes = du.transform_date_features_dict(dataframes, 'Date')
du.preview_dict_of_dfs(dataframes, n=2)

Preview of DataFrame for 2019Floor6:


,DataFrame Name,Date,z1_AC1(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),z1_S1(lux),z2_AC1(kW),z2_Light(kW),...,z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux),is_weekend,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,2019Floor6,2019-01-01 00:00:00,0.0,0.0,0.20,NaN,NaN,NaN,0.0,0.01,...,0.0,0.01,NaN,NaN,NaN,0,0.5,0.866025,0.781831,0.62349
1,2019Floor6,2019-01-01 00:01:00,0.0,0.0,0.16,NaN,NaN,NaN,0.0,0.01,...,0.0,0.01,NaN,NaN,NaN,0,0.5,0.866025,0.781831,0.62349




Preview of DataFrame for 2019Floor7:


,DataFrame Name,Date,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),...,z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux),is_weekend,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,2019Floor7,2019-01-01 00:00:00,0.0,0.0,0.01,0.0,0.0,0.13,NaN,NaN,...,0.0,0.08,NaN,NaN,NaN,0,0.5,0.866025,0.781831,0.62349
1,2019Floor7,2019-01-01 00:01:00,0.0,0.0,0.01,0.0,0.0,0.12,NaN,NaN,...,0.0,0.08,NaN,NaN,NaN,0,0.5,0.866025,0.781831,0.62349




Preview of DataFrame for 2019Floor5:


...

,DataFrame Name,Date,z1_Light(kW),z1_Plug(kW),z2_AC1(kW),z2_AC2(kW),z2_AC3(kW),z2_AC4(kW),z2_Light(kW),z2_Plug(kW),z3_Light(kW),z3_Plug(kW),z4_Light(kW),is_weekend,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,2018Floor1,2018-07-01 00:00:00,12.94,18.56,45.24,0.01,0.01,0.00,13.76,17.64,10.92,0.89,35.76,1,-0.5,-0.866025,-0.781831,0.62349
1,2018Floor1,2018-07-01 00:01:00,12.97,18.55,45.28,0.02,0.01,0.01,13.76,17.21,10.95,0.87,35.81,1,-0.5,-0.866025,-0.781831,0.62349




Preview of DataFrame for 2018Floor3:


,DataFrame Name,Date,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),...,z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux),is_weekend,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,2018Floor3,2018-07-01 00:00:00,0.0,0.0,0.0,0.07,0.0,0.03,28.44,60.67,...,0.0,0.02,26.88,71.16,0.0,1,-0.5,-0.866025,-0.781831,0.62349
1,2018Floor3,2018-07-01 00:01:00,0.0,0.0,0.0,0.07,0.0,0.03,28.44,60.67,...,0.0,0.02,26.88,71.16,0.0,1,-0.5,-0.866025,-0.781831,0.62349




Preview of DataFrame for 2018Floor2:


,DataFrame Name,Date,z1_AC1(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),z1_S1(lux),z2_AC1(kW),z2_AC2(kW),...,z4_Light(kW),z4_Plug(kW),z4_S1(degC),z4_S1(RH%),z4_S1(lux),is_weekend,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,2018Floor2,2018-07-01 00:00:00,0.0,0.01,0.15,28.36,58.93,0.0,1.08,0.0,...,0.0,0.0,NaN,NaN,NaN,1,-0.5,-0.866025,-0.781831,0.62349
1,2018Floor2,2018-07-01 00:01:00,0.0,0.01,0.15,28.36,58.91,0.0,1.07,0.0,...,0.0,0.0,NaN,NaN,NaN,1,-0.5,-0.866025,-0.781831,0.62349


In [7]:
du.nan_checker(dataframes)

,DataFrame Name,Date,day_of_week_cos,day_of_week_sin,is_weekend,month_cos,month_sin,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),...,z4_Plug(kW),z4_S1(RH%),z4_S1(degC),z4_S1(lux),z5_AC1(kW),z5_Light(kW),z5_Plug(kW),z5_S1(RH%),z5_S1(degC),z5_S1(lux)
0,No NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,Column not present,Column not present,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs
1,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs
2,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs
3,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs
4,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present
5,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs
6,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Column not present,Column not present,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present
7,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Column not present,Column not present,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs
8,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs
9,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs


# Create Target Column

## Create Total Consumption (kW) Column

In [8]:
# Assuming you have already loaded the dataframes into the `dataframes` dictionary
dataframes = du.add_total_consumption_columns(dataframes)
dataframes = du.convert_kw_to_kwh(dataframes)

In [9]:
dataframes = du.restructure_dataframes(dataframes)
du.nan_checker(dataframes)

,DataFrame Name,Date,day_of_week_cos,day_of_week_sin,is_weekend,month_cos,month_sin,total_consumption(kWh),z1_AC1(kWh),z1_AC2(kWh),...,z4_S1(degC),z4_S1(lux),z4_total_consumption(kWh),z5_AC1(kWh),z5_Light(kWh),z5_Plug(kWh),z5_S1(RH%),z5_S1(degC),z5_S1(lux),z5_total_consumption(kWh)
0,No NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs,Some NaNs,Some NaNs,No NaNs,Some NaNs,Column not present,...,Some NaNs,Some NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs
1,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs
2,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs
3,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs
4,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Column not present,Column not present,...,Column not present,Column not present,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present
5,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs
6,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Column not present,...,Some NaNs,Some NaNs,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present
7,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Column not present,...,Some NaNs,Some NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs
8,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs
9,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,No NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs


# Data Cleaning

## Imputing missing values with column averages by each dataframe

In [10]:
dataframes = du.impute_missing_values(dataframes)

DataFrame: 2019Floor6 - NaNs filled with column means
DataFrame: 2019Floor7 - NaNs filled with column means
DataFrame: 2019Floor5 - NaNs filled with column means
DataFrame: 2019Floor4 - NaNs filled with column means
DataFrame: 2019Floor1 - NaNs filled with column means
DataFrame: 2019Floor3 - NaNs filled with column means
DataFrame: 2019Floor2 - NaNs filled with column means
DataFrame: 2018Floor6 - NaNs filled with column means
DataFrame: 2018Floor7 - NaNs filled with column means
DataFrame: 2018Floor5 - NaNs filled with column means
DataFrame: 2018Floor4 - NaNs filled with column means
DataFrame: 2018Floor1 - NaNs filled with column means
DataFrame: 2018Floor3 - NaNs filled with column means
DataFrame: 2018Floor2 - NaNs filled with column means


In [11]:
du.nan_checker(dataframes)

,DataFrame Name,Date,day_of_week_cos,day_of_week_sin,is_weekend,month_cos,month_sin,total_consumption(kWh),z1_AC1(kWh),z1_AC2(kWh),...,z4_S1(degC),z4_S1(lux),z4_total_consumption(kWh),z5_AC1(kWh),z5_Light(kWh),z5_Plug(kWh),z5_S1(RH%),z5_S1(degC),z5_S1(lux),z5_total_consumption(kWh)
0,No NaNs,Some NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Column not present,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
1,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
2,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
3,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
4,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Column not present,Column not present,...,Column not present,Column not present,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present
5,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
6,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Column not present,...,No NaNs,No NaNs,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present
7,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,Column not present,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
8,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
9,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs


# Merging all dataframes by floor and adding column for source names

In [12]:
dataframes = du.merge_datasets_by_floor(dataframes)

for key, df in dataframes.items():
    print(f"DataFrame: {key}")


DataFrame: Floor6
DataFrame: Floor7
DataFrame: Floor5
DataFrame: Floor4
DataFrame: Floor1
DataFrame: Floor3
DataFrame: Floor2


In [13]:
dataframes = du.restructure_dataframes(dataframes)


In [14]:
du.check_column_names(dataframes)

DataFrame: Floor6
Columns: ['DataFrame Name', 'Date', 'z1_AC1(kWh)', 'z1_Light(kWh)', 'z1_Plug(kWh)', 'z1_S1(degC)', 'z1_S1(RH%)', 'z1_S1(lux)', 'z2_AC1(kWh)', 'z2_Light(kWh)', 'z2_Plug(kWh)', 'z2_S1(degC)', 'z2_S1(RH%)', 'z2_S1(lux)', 'z3_Light(kWh)', 'z3_Plug(kWh)', 'z4_AC1(kWh)', 'z4_AC2(kWh)', 'z4_AC3(kWh)', 'z4_AC4(kWh)', 'z4_Light(kWh)', 'z4_Plug(kWh)', 'z4_S1(degC)', 'z4_S1(RH%)', 'z4_S1(lux)', 'z5_AC1(kWh)', 'z5_Light(kWh)', 'z5_Plug(kWh)', 'z5_S1(degC)', 'z5_S1(RH%)', 'z5_S1(lux)', 'is_weekend', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 'total_consumption(kWh)', 'z1_total_consumption(kWh)', 'z2_total_consumption(kWh)', 'z3_total_consumption(kWh)', 'z4_total_consumption(kWh)', 'z5_total_consumption(kWh)']

DataFrame: Floor7
Columns: ['DataFrame Name', 'Date', 'z1_AC1(kWh)', 'z1_AC2(kWh)', 'z1_AC3(kWh)', 'z1_AC4(kWh)', 'z1_Light(kWh)', 'z1_Plug(kWh)', 'z1_S1(degC)', 'z1_S1(RH%)', 'z1_S1(lux)', 'z2_AC1(kWh)', 'z2_Light(kWh)', 'z2_Plug(kWh)', 'z2_S1(degC)', '

In [15]:
# from ingestion_utils import fetch_weather_data

# api_key = "BKB2RPATYBUZH9C3VXG83LZDW"
# location = "Bangkok,Thailand"
# start_date = "2019-01-01"
# end_date = "2019-12-31"

# weather_data = fetch_weather_data(location, start_date, end_date, api_key)

# if weather_data:
#     print(json.dumps(weather_data, indent=4))







In [16]:
#constraint temperature ac
# total_consumption_AC_Zone1 = B1consumptionAC1 + B1consumptionAC2 + B3consumptionAC3 + Ambient_temperature_Zone1 + Relative_humidity_Zone1 + Outdoor_temperature + Outdoor_relative_humidity
# find liner equeations for:

# convert all kW columns to kWh? -> yes

# add verbose options to functions

#name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations

In [17]:
# Example usage
file_path = 'weather_date/Bangkok,Thailand 2018-01-01 to 2019-12-31.csv'
daily_weather_data = wiu.fetch_weather_data_from_csv(file_path, amount_of_columns='essential')

if daily_weather_data is not None:
    display(daily_weather_data.head())

,datetime,temp,tempmax,tempmin,humidity,precip,windspeed,uvindex,conditions
0,2018-01-01,26.1,30.3,22.0,0.666,0.0,17.9,5,Partially cloudy
1,2018-01-02,26.7,30.9,22.2,0.656,0.0,18.0,5,Partially cloudy
2,2018-01-03,27.5,31.0,24.2,0.650,0.0,12.5,5,Partially cloudy
3,2018-01-04,28.4,33.0,24.0,0.640,0.0,13.0,8,Partially cloudy
4,2018-01-05,29.5,34.0,25.0,0.620,0.0,10.3,8,Partially cloudy


In [18]:

approx_hourly_weather = wiu.approximate_hourly_weather(daily_weather_data)
display(approx_hourly_weather.head(24))  # Print the first 24 hours to see the approximation

,datetime,temp,humidity,precip,windspeed,uvindex,conditions
0,2018-01-01 00:00:00,22.56,0.67,0.0,17.90,5.0,Partially cloudy
1,2018-01-01 01:00:00,22.14,0.67,0.0,17.90,5.0,Partially cloudy
2,2018-01-01 02:00:00,22.00,0.67,0.0,17.91,5.0,Partially cloudy
3,2018-01-01 03:00:00,22.14,0.66,0.0,17.91,5.0,Partially cloudy
4,2018-01-01 04:00:00,22.56,0.66,0.0,17.92,5.0,Partially cloudy
5,2018-01-01 05:00:00,23.22,0.66,0.0,17.92,5.0,Partially cloudy
6,2018-01-01 06:00:00,24.08,0.66,0.0,17.92,5.0,Partially cloudy
7,2018-01-01 07:00:00,25.08,0.66,0.0,17.93,5.0,Partially cloudy
8,2018-01-01 08:00:00,26.15,0.66,0.0,17.93,5.0,Partially cloudy
9,2018-01-01 09:00:00,27.22,0.66,0.0,17.94,5.0,Partially cloudy


In [19]:
dataframes = du.aggregate_dataframes(dataframes)
du.check_column_names(dataframes)                                  

DataFrame: Floor6
Columns: ['Date', 'DataFrame Name', 'z1_AC1(kWh)', 'z1_Light(kWh)', 'z1_Plug(kWh)', 'z1_S1(degC)', 'z1_S1(RH%)', 'z1_S1(lux)', 'z2_AC1(kWh)', 'z2_Light(kWh)', 'z2_Plug(kWh)', 'z2_S1(degC)', 'z2_S1(RH%)', 'z2_S1(lux)', 'z3_Light(kWh)', 'z3_Plug(kWh)', 'z4_AC1(kWh)', 'z4_AC2(kWh)', 'z4_AC3(kWh)', 'z4_AC4(kWh)', 'z4_Light(kWh)', 'z4_Plug(kWh)', 'z4_S1(degC)', 'z4_S1(RH%)', 'z4_S1(lux)', 'z5_AC1(kWh)', 'z5_Light(kWh)', 'z5_Plug(kWh)', 'z5_S1(degC)', 'z5_S1(RH%)', 'z5_S1(lux)', 'is_weekend', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 'total_consumption(kWh)', 'z1_total_consumption(kWh)', 'z2_total_consumption(kWh)', 'z3_total_consumption(kWh)', 'z4_total_consumption(kWh)', 'z5_total_consumption(kWh)']

DataFrame: Floor7
Columns: ['Date', 'DataFrame Name', 'z1_AC1(kWh)', 'z1_AC2(kWh)', 'z1_AC3(kWh)', 'z1_AC4(kWh)', 'z1_Light(kWh)', 'z1_Plug(kWh)', 'z1_S1(degC)', 'z1_S1(RH%)', 'z1_S1(lux)', 'z2_AC1(kWh)', 'z2_Light(kWh)', 'z2_Plug(kWh)', 'z2_S1(degC)', '

In [20]:
dataframes = du.calculate_total_ac_consumption_by_zone(dataframes)
du.check_column_names(dataframes) 

DataFrame: Floor6
Columns: ['Date', 'DataFrame Name', 'z1_AC1(kWh)', 'z1_Light(kWh)', 'z1_Plug(kWh)', 'z1_S1(degC)', 'z1_S1(RH%)', 'z1_S1(lux)', 'z2_AC1(kWh)', 'z2_Light(kWh)', 'z2_Plug(kWh)', 'z2_S1(degC)', 'z2_S1(RH%)', 'z2_S1(lux)', 'z3_Light(kWh)', 'z3_Plug(kWh)', 'z4_AC1(kWh)', 'z4_AC2(kWh)', 'z4_AC3(kWh)', 'z4_AC4(kWh)', 'z4_Light(kWh)', 'z4_Plug(kWh)', 'z4_S1(degC)', 'z4_S1(RH%)', 'z4_S1(lux)', 'z5_AC1(kWh)', 'z5_Light(kWh)', 'z5_Plug(kWh)', 'z5_S1(degC)', 'z5_S1(RH%)', 'z5_S1(lux)', 'is_weekend', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 'total_consumption(kWh)', 'z1_total_consumption(kWh)', 'z2_total_consumption(kWh)', 'z3_total_consumption(kWh)', 'z4_total_consumption(kWh)', 'z5_total_consumption(kWh)', 'z4_total_AC_consumption(kW)', 'z2_total_AC_consumption(kW)', 'z1_total_AC_consumption(kW)', 'z5_total_AC_consumption(kW)']

DataFrame: Floor7
Columns: ['Date', 'DataFrame Name', 'z1_AC1(kWh)', 'z1_AC2(kWh)', 'z1_AC3(kWh)', 'z1_AC4(kWh)', 'z1_Light(kWh)',

# Getting Constraint Equations

In [21]:
constraints = mu.store_constraints(dataframes)

for name, constraint in constraints.items():
    print(f"DataFrame: {name}")
    print(f"Total Floor Consumption: {constraint['total_floor_consumption']}")
    print(f"Total Zone Consumption: {constraint['total_zone_consumption']}")
    print(f"Total AC Consumption by Zone: {constraint['total_ac_consumption_by_zone']}")

DataFrame: Floor6
Total Floor Consumption: total_consumption(kWh) + z1_AC1(kWh) + z1_Light(kWh) + z1_Plug(kWh) + z1_total_AC_consumption(kW) + z1_total_consumption(kWh) + z2_AC1(kWh) + z2_Light(kWh) + z2_Plug(kWh) + z2_total_AC_consumption(kW) + z2_total_consumption(kWh) + z3_Light(kWh) + z3_Plug(kWh) + z3_total_consumption(kWh) + z4_AC1(kWh) + z4_AC2(kWh) + z4_AC3(kWh) + z4_AC4(kWh) + z4_Light(kWh) + z4_Plug(kWh) + z4_total_AC_consumption(kW) + z4_total_consumption(kWh) + z5_AC1(kWh) + z5_Light(kWh) + z5_Plug(kWh) + z5_total_AC_consumption(kW) + z5_total_consumption(kWh)
Total Zone Consumption: {}
Total AC Consumption by Zone: {}
DataFrame: Floor7
Total Floor Consumption: total_consumption(kWh) + z1_AC1(kWh) + z1_AC2(kWh) + z1_AC3(kWh) + z1_AC4(kWh) + z1_Light(kWh) + z1_Plug(kWh) + z1_total_AC_consumption(kW) + z1_total_consumption(kWh) + z2_AC1(kWh) + z2_Light(kWh) + z2_Plug(kWh) + z2_total_AC_consumption(kW) + z2_total_consumption(kWh) + z3_Light(kWh) + z3_Plug(kWh) + z3_total_consu

In [ ]:
# test